In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [4]:
df_credits = pd.read_csv("tmdb_5000_credits.csv", sep = ",")
df_movies = pd.read_csv("tmdb_5000_movies.csv", sep = ",")
df_imdb = pd.read_csv("imdb_data.csv")

In [5]:
used_col = ['budget', 'genres', 'id', 'popularity', 'revenue', 'runtime', 'title', 'num_critic_for_reviews', 'duration'
            , 'actor_1_facebook_likes', 'director_facebook_likes', 'actor_3_facebook_likes', 'gross', 'cast_total_facebook_likes'
            , 'facenumber_in_poster', 'num_user_for_reviews', 'actor_2_facebook_likes', 'imdb_score', 'aspect_ratio', "movie_facebook_likes"]

In [6]:
df_imdb_nn = df_imdb.loc[:, used_col]

In [7]:
df_imdb_nn.set_index("id", inplace = True)

In [8]:
from ast import literal_eval
df_imdb_nn.loc[:, 'genres'] = df_imdb_nn.loc[:, 'genres'].apply(literal_eval)

In [11]:
df_imdb_nn.loc[:, 'genres'] = df_imdb_nn.loc[:, 'genres'].apply(get_list)

In [12]:
df_imdb_nn.loc[:, "genres"] = df_imdb_nn.loc[:, 'genres'].apply(clean_data)

In [13]:
df_imdb_nn.loc[:, 'genres_str'] = df_imdb_nn.loc[:, 'genres'].apply(lambda x : " ".join(x))

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv.fit(df_imdb_nn['genres_str'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [15]:
df_genres = pd.DataFrame(index = df_imdb_nn.index, columns=cv.get_feature_names())

In [16]:
df_genres_t = df_genres.T

In [17]:
for i in df_genres.index:
    df_genres_t.loc[:, i] = cv.transform([df_imdb_nn["genres_str"][i]]).toarray()[0]

In [21]:
nn_feature = ['budget', 'popularity', 'revenue', 'num_critic_for_reviews', 'duration', 'actor_1_facebook_likes',
       'director_facebook_likes', 'actor_3_facebook_likes', 'gross', 'cast_total_facebook_likes', 'facenumber_in_poster',
       'num_user_for_reviews', 'actor_2_facebook_likes', 'aspect_ratio', 'movie_facebook_likes']
X = df_imdb_nn[nn_feature]
X = X.merge(df_genres, left_index = True, right_index = True, how = "left")

y = df_imdb_nn['imdb_score']

In [22]:
score_dict = {}
count = 1
for i in np.arange(0.1, 10.1, step = 0.1):
    i = round(i, 1)
    score_dict[str(i)] = count
    count += 1

y = y.apply(lambda x : score_dict[str(x)])

In [23]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

std_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

X_std_scale = std_scaler.fit_transform(X)

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

X_std_train, X_std_test, y_std_train, y_std_test = train_test_split(X_std_scale, y, test_size = 0.2, random_state = 42)
X_std_train, X_std_val, y_std_train, y_std_val = train_test_split(X_std_train, y_std_train, test_size = 0.2, random_state = 42)

In [25]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, 100)
y_val = np_utils.to_categorical(y_val, 100)
y_test = np_utils.to_categorical(y_test, 100)

y_std_train = np_utils.to_categorical(y_std_train, 100)
y_std_val = np_utils.to_categorical(y_std_val, 100)
y_std_test = np_utils.to_categorical(y_std_test, 100)

Using TensorFlow backend.


# Model with Sparse AE pre-training on Standarized Data

In [26]:
K = keras.backend
kl_divergence = keras.losses.kullback_leibler_divergence

class KLDivergenceRegularizer(keras.regularizers.Regularizer):
    def __init__(self, weight, target=0.1):
        self.weight = weight
        self.target = target
    def __call__(self, inputs):
        mean_activities = K.mean(inputs, axis=0)
        return self.weight * (
            kl_divergence(self.target, mean_activities) +
            kl_divergence(1. - self.target, 1. - mean_activities))

In [30]:
from tensorflow.keras.models import Model
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(42)
np.random.seed(42)

kld_reg = KLDivergenceRegularizer(weight=0.05, target=0.1)
sparse_l1_encoder = keras.models.Sequential([
    keras.layers.Dense(200, input_dim = 35, activation = "relu"),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(25, activation="relu", activity_regularizer=kld_reg)  # Alternatively, you could add
                                                  # activity_regularizer=keras.regularizers.l1(1e-3)
                                                  # to the previous layer.
])
sparse_l1_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="relu", input_shape=[25]),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dense(35, activation="relu")
])

sparse_l1_ae = keras.models.Sequential([sparse_l1_encoder, sparse_l1_decoder])
sparse_l1_ae.compile(loss="mse", optimizer="adam")

history = sparse_l1_ae.fit(X_std_train, X_std_train, batch_size = 32, epochs=64,
                           validation_data=[X_std_val, X_std_val])


Train on 2843 samples, validate on 711 samples
Epoch 1/64
2843/2843 [==============================] - 1s 401us/sample - loss: 0.7868 - val_loss: 0.5670
Epoch 2/64
2843/2843 [==============================] - 0s 88us/sample - loss: 0.5386 - val_loss: 0.4875
Epoch 3/64
2843/2843 [==============================] - 0s 88us/sample - loss: 0.4581 - val_loss: 0.4586
Epoch 4/64
2843/2843 [==============================] - 0s 82us/sample - loss: 0.4265 - val_loss: 0.4446
Epoch 5/64
2843/2843 [==============================] - 0s 88us/sample - loss: 0.4106 - val_loss: 0.4340
Epoch 6/64
2843/2843 [==============================] - 0s 88us/sample - loss: 0.4035 - val_loss: 0.4268
Epoch 7/64
2843/2843 [==============================] - 0s 88us/sample - loss: 0.3973 - val_loss: 0.4246
Epoch 8/64
2843/2843 [==============================] - 0s 104us/sample - loss: 0.3952 - val_loss: 0.4205
Epoch 9/64
2843/2843 [==============================] - 0s 88us/sample - loss: 0.3933 - val_loss: 0.4233
Epoch 

In [32]:
dnn_clf = keras.models.Sequential([
    keras.layers.Dense(10, input_dim=25, activation = "relu"),
    keras.layers.Dense(64, activation = "relu"),
    keras.layers.Dense(100, activation = "softmax")
])

ae_dnn_clf = keras.models.Sequential([sparse_l1_encoder, dnn_clf])
ae_dnn_clf.compile(loss = "categorical_crossentropy", optimizer = "Adam", metrics = ["accuracy"])

ae_dnn_clf.fit(X_std_train, y_std_train, batch_size = 32, epochs = 32,
              validation_data = [X_std_val, y_std_val])

Train on 2843 samples, validate on 711 samples
Epoch 1/32
2843/2843 [==============================] - 1s 412us/sample - loss: 4.3156 - accuracy: 0.0369 - val_loss: 3.9570 - val_accuracy: 0.0478
Epoch 2/32
2843/2843 [==============================] - 0s 88us/sample - loss: 3.8694 - accuracy: 0.0440 - val_loss: 3.8218 - val_accuracy: 0.0549
Epoch 3/32
2843/2843 [==============================] - 0s 88us/sample - loss: 3.7261 - accuracy: 0.0489 - val_loss: 3.7087 - val_accuracy: 0.0549
Epoch 4/32
2843/2843 [==============================] - 0s 93us/sample - loss: 3.5998 - accuracy: 0.0658 - val_loss: 3.6149 - val_accuracy: 0.0577
Epoch 5/32
2843/2843 [==============================] - 0s 88us/sample - loss: 3.5032 - accuracy: 0.0823 - val_loss: 3.5941 - val_accuracy: 0.0506
Epoch 6/32
2843/2843 [==============================] - 0s 88us/sample - loss: 3.4334 - accuracy: 0.0904 - val_loss: 3.5961 - val_accuracy: 0.0689
Epoch 7/32
2843/2843 [==============================] - 0s 93us/sample

In [39]:
y_pred = ae_dnn_clf.predict(X_std_test)
score = ae_dnn_clf.evaluate(X_std_test, y_std_test)

df_testing_result = pd.DataFrame(index = range(len(y_pred)), columns = ["Predict", "Actual"])
for i in range(len(y_pred)):
    df_testing_result.loc[i, "Predict"] = np.argmax(y_pred[i])
    df_testing_result.loc[i, "Actual"] = np.argmax(y_test[i])
    
df_testing_result.loc[:, "Error between +1.0 ~ -1.0"] = df_testing_result.apply(lambda x : "True" if abs(x[0] - x[1]) <=  else "false", axis = 1)

889/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [40]:
score_interval = len(df_testing_result[df_testing_result['Error between +1.0 ~ -1.0'] == 'True']) / len(df_testing_result)

print(f'accuracy : {score[1]}')
print(f'accuracy between +1.0 ~ -1.0 : {score_interval}')

accuracy : 0.043869517743587494
accuracy between +1.0 ~ -1.0 : 0.9471316085489314


# Model on Original Data

In [39]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import losses

model = Sequential()
model.add(Dense(20, input_dim=35, activation = "relu"))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics = ["accuracy"])

In [40]:
model.fit(X_train, y_train, epochs=32, batch_size=100, validation_data=(X_val, y_val))

W1220 17:05:19.291732  8268 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Train on 2843 samples, validate on 711 samples
Epoch 1/32
2843/2843 [==============================] - 1s 226us/sample - loss: 15.5282 - accuracy: 0.0338 - val_loss: 15.2807 - val_accuracy: 0.0492
Epoch 2/32
2843/2843 [==============================] - 0s 74us/sample - loss: 15.4666 - accuracy: 0.0366 - val_loss: 15.2463 - val_accuracy: 0.0478
Epoch 3/32
2843/2843 [==============================] - 0s 65us/sample - loss: 15.3936 - accuracy: 0.0422 - val_loss: 15.3034 - val_accuracy: 0.0464
Epoch 4/32
2843/2843 [==============================] - 0s 65us/sample - loss: 15.3895 - accuracy: 0.0426 - val_loss: 15.2249 - val_accuracy: 0.0520
Epoch 5/32
2843/2843 [==============================] - 0s 59us/sample - loss: 15.4579 - accuracy: 0.0390 - val_loss: 15.4152 - val_accuracy: 0.0422
Epoch 6/32
2843/2843 [==============================] - 0s 58us/sample - loss: 15.5408 - accuracy: 0.0341 - val_loss: 15.3618 - val_accuracy: 0.0450
Epoch 7/32
2843/2843 [==============================] - 0s

# Model on Standarized Data

In [148]:
model_std = Sequential()
model_std.add(Dense(20, input_dim=35, activation = "relu"))
model_std.add(Dense(64, activation='relu'))
model_std.add(Dense(128, activation='relu'))
model_std.add(Dense(64, activation='relu'))
model_std.add(Dense(100, activation="softmax"))
model_std.compile(loss="categorical_crossentropy", optimizer='Adam', metrics = ["accuracy"])

In [149]:
model_std.fit(X_std_train, y_std_train, epochs=32, batch_size=100, validation_data=(X_std_val, y_std_val))

Train on 2843 samples, validate on 711 samples
Epoch 1/32
2843/2843 [==============================] - 2s 568us/sample - loss: 4.4436 - accuracy: 0.0229 - val_loss: 4.1542 - val_accuracy: 0.0267
Epoch 2/32
2843/2843 [==============================] - 0s 37us/sample - loss: 3.9457 - accuracy: 0.0376 - val_loss: 3.8537 - val_accuracy: 0.0338
Epoch 3/32
2843/2843 [==============================] - 0s 39us/sample - loss: 3.7816 - accuracy: 0.0521 - val_loss: 3.7987 - val_accuracy: 0.0422
Epoch 4/32
2843/2843 [==============================] - 0s 39us/sample - loss: 3.7143 - accuracy: 0.0492 - val_loss: 3.7561 - val_accuracy: 0.0422
Epoch 5/32
2843/2843 [==============================] - 0s 37us/sample - loss: 3.6572 - accuracy: 0.0594 - val_loss: 3.7295 - val_accuracy: 0.0323
Epoch 6/32
2843/2843 [==============================] - 0s 39us/sample - loss: 3.5929 - accuracy: 0.0711 - val_loss: 3.6867 - val_accuracy: 0.0394
Epoch 7/32
2843/2843 [==============================] - 0s 46us/sample

In [ ]:
y_pred_std = model_std.predict(X_std_test)
score_std = model_std.evaluate(X_std_test, y_std_test)

df_result_std = pd.DataFrame(index = range(len(y_pred)), columns = ["Predict", "Actual"])
for i in range(len(y_pred)):
    df_result_std.loc[i, "Predict"] = np.argmax(y_pred_std[i])
    df_result_std.loc[i, "Actual"] = np.argmax(y_test[i])
    
df_result_std.loc[:, "Error between +1.0 ~ -1.0"] = df_result_std.apply(lambda x : "True" if abs(x[0] - x[1]) <= 10 else "false", axis = 1)

In [ ]:
score_interval_std = len(df_testing_result[df_testing_result['Error between +1.0 ~ -1.0'] == 'True']) / len(df_testing_result)

print(f'accuracy : {score_std[1]}')
print(f'accuracy between +1.0 ~ -1.0 : {score_interval_std}')